# 4. Orquestación de Agentes con CrewAI

## Objetivos de Aprendizaje
- Entender el concepto de agentes colaborativos y los roles en un "equipo" (Crew).
- Aprender los componentes clave de CrewAI: `Agent`, `Task`, `Tool` y `Crew`.
- Definir un equipo de agentes (un investigador y un escritor) para realizar una tarea compleja.
- Ejecutar el `Crew` y observar cómo los agentes colaboran y se pasan el trabajo entre ellos.
- **Comprender las configuraciones específicas necesarias para usar CrewAI con GitHub Models API**.

## ¿Qué es CrewAI y por qué usarlo?

Mientras que LangChain proporciona los bloques de construcción fundamentales para crear un agente, **CrewAI** se especializa en la **orquestación de múltiples agentes autónomos**. La idea central es que, para resolver tareas complejas, es más eficiente tener un equipo de agentes especializados que colaboren, en lugar de un solo agente que intente hacerlo todo.

**Analogía:** Piensa en una agencia de marketing. No tienes una sola persona que es experta en investigación, redacción, diseño y redes sociales. Tienes un equipo donde cada miembro tiene un rol claro. CrewAI aplica este concepto a los agentes de IA.

**Ventajas de CrewAI:**
- **Roles Especializados**: Permite definir agentes con roles, objetivos (`goal`) e historias de fondo (`backstory`) específicas, lo que los hace más efectivos en su nicho.
- **Colaboración Autónoma**: Los agentes pueden delegar tareas entre ellos de forma autónoma.
- **Procesos Secuenciales y Jerárquicos**: Soporta flujos de trabajo donde las tareas se completan en un orden específico.
- **Claridad y Estructura**: El código es muy declarativo y fácil de leer, ya que se centra en definir el equipo y sus responsabilidades.

## Relación entre CrewAI y LangChain

CrewAI **utiliza LangChain internamente** para manejar los LLMs y las herramientas. Esto significa que:

1. **CrewAI** se encarga de la orquestación de agentes (el "director de orquesta")
2. **LangChain** proporciona la interfaz con los modelos y herramientas (los "instrumentos")

Esta relación requiere configuraciones específicas que veremos en este notebook.

### 1. Instalación y Configuración

In [ ]:
!pip install crewai langchain-openai openai wikipedia -q

### 2. Configuración Especial para GitHub Models API

**⚠️ IMPORTANTE:** Esta es la parte crítica que causa problemas si no se configura correctamente.

**El Problema:**
- CrewAI utiliza LangChain internamente
- LangChain busca las variables de entorno `OPENAI_API_KEY` y `OPENAI_API_BASE`
- Nosotros tenemos `GITHUB_TOKEN` y `OPENAI_BASE_URL`
- Necesitamos "mapear" nuestras variables a las que LangChain espera

**La Solución:**
Configuramos las variables de entorno que LangChain espera, usando nuestros valores de GitHub Models API.

In [27]:
import os

# 🔧 CONFIGURACIÓN CRÍTICA: Mapear variables de entorno para LangChain
# CrewAI → LangChain → GitHub Models API

# LangChain espera estas variables específicas:
os.environ["OPENAI_API_BASE"] = os.environ.get("OPENAI_BASE_URL", "")
os.environ["OPENAI_API_KEY"] = os.environ.get("GITHUB_TOKEN", "")

# Verificar que las variables estén configuradas
print("🔍 Verificando configuración:")
print(f"OPENAI_BASE_URL: {os.environ.get('OPENAI_BASE_URL', 'No configurado')}")
print(f"GITHUB_TOKEN: {'✅ Configurado' if os.environ.get('GITHUB_TOKEN') else '❌ No configurado'}")
print(f"OPENAI_API_BASE: {os.environ.get('OPENAI_API_BASE', 'No configurado')}")
print(f"OPENAI_API_KEY: {'✅ Configurado' if os.environ.get('OPENAI_API_KEY') else '❌ No configurado'}")

print("\n✅ Variables de entorno mapeadas correctamente para LangChain.")

🔍 Verificando configuración:
OPENAI_BASE_URL: https://models.inference.ai.azure.com
GITHUB_TOKEN: ✅ Configurado
OPENAI_API_BASE: https://models.inference.ai.azure.com
OPENAI_API_KEY: ✅ Configurado

✅ Variables de entorno mapeadas correctamente para LangChain.


### 3. Configuración del LLM

Ahora configuramos el LLM usando LangChain. Gracias a la configuración anterior, LangChain automáticamente usará nuestras variables de entorno mapeadas.

In [28]:
import wikipedia
from langchain_openai import ChatOpenAI

# Configurar el idioma de Wikipedia
wikipedia.set_lang('es')

# 🧠 Configurar el LLM con LangChain
# Nota: No necesitamos pasar explícitamente las API keys aquí
# porque LangChain las lee automáticamente de las variables de entorno
try:
    llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0
    )
    
    # Probar que el LLM funciona
    test_response = llm.invoke("Hola, ¿funcionas correctamente?")
    print(f"✅ LLM configurado y probado exitosamente.")
    print(f"📝 Respuesta de prueba: {test_response.content[:50]}...")
    
except Exception as e:
    print(f"❌ Error configurando el LLM: {e}")
    print("💡 Verifica que OPENAI_BASE_URL y GITHUB_TOKEN estén configurados correctamente.")
    llm = None

✅ LLM configurado y probado exitosamente.
📝 Respuesta de prueba: ¡Hola! Sí, funciono correctamente. 😊 ¿En qué puedo...


### 4. Definición de Herramientas con CrewAI

**⚠️ IMPORTANTE:** CrewAI requiere un enfoque específico para las herramientas.

**El Problema:**
- LangChain usa el decorador `@tool` para definir herramientas
- CrewAI requiere que las herramientas hereden de `BaseTool`
- Mezclar ambos enfoques causa errores

**La Solución:**
Usar `BaseTool` de `crewai_tools` para crear herramientas compatibles con CrewAI.

In [29]:
from crewai_tools import BaseTool

# 🔧 HERRAMIENTA CORREGIDA: Usar BaseTool en lugar de @tool
class WikipediaSearchTool(BaseTool):
    name: str = "Wikipedia Search Tool"
    description: str = "Busca en Wikipedia un tema y devuelve un resumen detallado. Es ideal para obtener información sobre personas, lugares, conceptos históricos y científicos."
    
    def _run(self, query: str) -> str:
        """Ejecuta la búsqueda en Wikipedia"""
        try:
            # Configurar el idioma de Wikipedia
            wikipedia.set_lang("es")
            # Devolver un resumen detallado para que el escritor tenga más material
            return wikipedia.summary(query, sentences=5)
        except wikipedia.exceptions.PageError:
            return f"No se encontró ninguna página para '{query}'. Intenta con un término más específico."
        except wikipedia.exceptions.DisambiguationError as e:
            return f"La búsqueda para '{query}' es ambigua. Opciones disponibles: {e.options[:3]}. Especifica cuál te interesa."
        except Exception as e:
            return f"Error al buscar en Wikipedia: {str(e)}"

# Crear la instancia de la herramienta
wikipedia_tool = WikipediaSearchTool()
tools = [wikipedia_tool]

# Probar la herramienta
try:
    test_result = wikipedia_tool._run("Albert Einstein")
    print("✅ Herramienta de Wikipedia configurada y probada exitosamente.")
    print(f"📝 Resultado de prueba: {test_result[:100]}...")
except Exception as e:
    print(f"❌ Error probando la herramienta: {e}")

print(f"\n🔧 {len(tools)} herramienta(s) disponible(s) para los agentes.")

✅ Herramienta de Wikipedia configurada y probada exitosamente.
📝 Resultado de prueba: Albert Einstein pronunciación en alemán: /ˈalbɐt ˈaɪnʃtaɪn/ ();[2]​ (Ulm, Imperio alemán, 14 de marz...

🔧 1 herramienta(s) disponible(s) para los agentes.


### 5. Creación del Equipo de Agentes (Crew)

Ahora definimos nuestro equipo de agentes especializados:

1. **Investigador (Researcher)**: Busca información detallada usando Wikipedia
2. **Escritor (Writer)**: Transforma la información en una biografía bien redactada

**⚠️ IMPORTANTE:** Otro error común es el parámetro `verbose` en `Crew`.

In [30]:
from crewai import Agent, Task, Crew, Process

# 🕵️ Agente 1: El Investigador
researcher = Agent(
    role="Investigador Senior",
    goal="Encontrar información completa y precisa sobre personas históricas, científicos y figuras importantes utilizando fuentes confiables.",
    backstory="""Eres un investigador académico con años de experiencia en la búsqueda de información histórica y científica. 
    Tu especialidad es encontrar datos precisos y relevantes en Wikipedia y otras fuentes confiables. 
    Te enorgulleces de la exactitud de tu trabajo y siempre proporcionas contexto histórico relevante. 
    No escribes biografías completas, tu trabajo es recopilar los datos más importantes y precisos.""",
    tools=tools,
    llm=llm,
    verbose=True,
    allow_delegation=False  # Este agente no delega trabajo
)

# ✍️ Agente 2: El Escritor
writer = Agent(
    role="Escritor de Biografías",
    goal="Crear biografías atractivas, bien estructuradas y fáciles de leer basadas en la información proporcionada por el investigador.",
    backstory="""Eres un escritor profesional especializado en biografías y divulgación científica. 
    Tu habilidad única es transformar datos técnicos y históricos en narrativas cautivadoras que son 
    tanto informativas como accesibles para el público general. 
    Tienes un don especial para destacar los aspectos más interesantes de la vida de las personas 
    y presentar sus logros de manera inspiradora.""",
    llm=llm,
    verbose=True,
    allow_delegation=False
)

print("✅ Agentes creados exitosamente:")
print(f"🕵️ {researcher.role}")
print(f"✍️ {writer.role}")

✅ Agentes creados exitosamente:
🕵️ Investigador Senior
✍️ Escritor de Biografías


### 6. Definición de Tareas

Las tareas definen exactamente qué debe hacer cada agente y cómo se relacionan entre ellas.

In [31]:
# 🔍 Tarea 1: Investigación
research_task = Task(
    description="""Busca información detallada sobre Marie Curie en Wikipedia. 
    Enfócate en:
    - Sus descubrimientos científicos más importantes
    - Su impacto en la ciencia y la sociedad
    - Datos biográficos clave (fechas, lugares, educación)
    - Sus premios y reconocimientos
    - Su legado científico
    
    Proporciona información precisa y bien organizada que el escritor pueda usar.""",
    expected_output="Un resumen detallado de 4-6 párrafos con los datos más importantes sobre la vida, descubrimientos y legado de Marie Curie.",
    agent=researcher
)

# ✍️ Tarea 2: Escritura
write_task = Task(
    description="""Usando la información recopilada por el investigador, escribe una biografía cautivadora de Marie Curie.
    
    Requisitos:
    - Mínimo 5 párrafos bien estructurados
    - Estilo atractivo y accesible
    - Incluir sus logros más importantes
    - Destacar su impacto en la ciencia
    - Formato Markdown con encabezados apropiados
    - Tono inspirador pero preciso
    
    La biografía debe ser educativa e inspiradora para lectores de todas las edades.""",
    expected_output="Una biografía completa en formato Markdown, bien estructurada y atractiva, de al menos 5 párrafos.",
    agent=writer,
    context=[research_task]  # Esta tarea depende del resultado de la investigación
)

print("✅ Tareas definidas exitosamente:")
print(f"🔍 Tarea de investigación: {research_task.description[:50]}...")
print(f"✍️ Tarea de escritura: {write_task.description[:50]}...")

✅ Tareas definidas exitosamente:
🔍 Tarea de investigación: Busca información detallada sobre Marie Curie en W...
✍️ Tarea de escritura: Usando la información recopilada por el investigad...


### 7. Ensamblaje del Equipo (Crew)

**⚠️ CONFIGURACIÓN CORREGIDA:** El parámetro `verbose` debe ser boolean, no entero.

In [32]:
# 🎯 CONFIGURACIÓN CORREGIDA: verbose debe ser boolean
crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, write_task],
    process=Process.sequential,  # Las tareas se ejecutan en orden
    verbose=True  # ✅ CORRECTO: boolean, no entero (verbose=2 causaría error)
)

print("✅ Equipo (Crew) ensamblado exitosamente:")
print(f"👥 {len(crew.agents)} agentes en el equipo")
print(f"📋 {len(crew.tasks)} tareas definidas")
print(f"🔄 Proceso: {crew.process}")
print(f"🔊 Verbose: {crew.verbose}")

2025-07-03 13:14:08,669 - 140704377686976 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


✅ Equipo (Crew) ensamblado exitosamente:
👥 2 agentes en el equipo
📋 2 tareas definidas
🔄 Proceso: Process.sequential
🔊 Verbose: True


### 8. Ejecución del Crew

¡Ahora viene la magia! Ejecutamos el crew y observamos cómo los agentes colaboran.

In [33]:
# 🚀 Ejecutar el crew
if llm is None:
    print("❌ No se puede ejecutar el crew sin un LLM configurado.")
    print("💡 Verifica la configuración de las variables de entorno en las celdas anteriores.")
else:
    try:
        print("🚀 Iniciando ejecución del crew...")
        print("📝 Observa cómo los agentes colaboran paso a paso:\n")
        
        # Ejecutar el crew
        result = crew.kickoff()
        
        print("\n" + "="*80)
        print("🏁 RESULTADO FINAL DEL CREW")
        print("="*80)
        print(result)
        
    except Exception as e:
        print(f"❌ Error durante la ejecución del crew: {e}")
        print("\n🔧 Posibles soluciones:")
        print("1. Verifica que GITHUB_TOKEN esté configurado correctamente")
        print("2. Asegúrate de que OPENAI_BASE_URL esté configurado")
        print("3. Confirma que tu token tenga permisos para usar GitHub Models")
        print("4. Verifica que no hayas mezclado @tool con BaseTool")
        print("5. Asegúrate de que verbose=True (no verbose=2)")
        
        # Mostrar información de debugging
        print("\n🐛 Información de debugging:")
        import traceback
        traceback.print_exc()

🚀 Iniciando ejecución del crew...
📝 Observa cómo los agentes colaboran paso a paso:

# Agent: Investigador Senior
## Task: Busca información detallada sobre Marie Curie en Wikipedia. 
    Enfócate en:
    - Sus descubrimientos científicos más importantes
    - Su impacto en la ciencia y la sociedad
    - Datos biográficos clave (fechas, lugares, educación)
    - Sus premios y reconocimientos
    - Su legado científico
    
    Proporciona información precisa y bien organizada que el escritor pueda usar.


# Agent: Investigador Senior
## Thought: Para proporcionar un resumen detallado y preciso sobre Marie Curie, utilizaré la herramienta de búsqueda en Wikipedia para recopilar información sobre sus descubrimientos científicos, impacto, datos biográficos, premios y legado.
## Using tool: Wikipedia Search Tool
## Tool Input: 
"{\"query\": \"Marie Curie\"}"
## Tool Output: 
Maria Salomea Skłodowska-Curie,[A]​[B]​ más conocida como Marie Curie[C]​[B]​ o Madame Curie (Varsovia, 7 de noviembr

## 🎓 Resumen de Configuraciones Críticas

### Problemas Comunes y Sus Soluciones

| **Problema** | **Síntoma** | **Solución** |
|-------------|-------------|-------------|
| **Variables de entorno** | `AuthenticationError: Incorrect API key` | Mapear `GITHUB_TOKEN` → `OPENAI_API_KEY` y `OPENAI_BASE_URL` → `OPENAI_API_BASE` |
| **Herramientas incompatibles** | `'Tool' object is not callable` | Usar `BaseTool` en lugar de `@tool` |
| **Parámetro verbose** | `ValidationError: Input should be a valid boolean` | Usar `verbose=True` en lugar de `verbose=2` |
| **LLM no configurado** | `llm is None` | Verificar que las variables de entorno estén configuradas |

### Configuración Correcta para GitHub Models API

```python
# 1. Mapear variables de entorno
os.environ["OPENAI_API_BASE"] = os.environ.get("OPENAI_BASE_URL", "")
os.environ["OPENAI_API_KEY"] = os.environ.get("GITHUB_TOKEN", "")

# 2. Configurar LLM sin parámetros explícitos
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# 3. Usar BaseTool para herramientas
class MiTool(BaseTool):
    name: str = "Mi Herramienta"
    description: str = "Descripción de la herramienta"
    def _run(self, query: str) -> str:
        return "resultado"

# 4. Configurar Crew con verbose boolean
crew = Crew(
    agents=[agent1, agent2],
    tasks=[task1, task2],
    process=Process.sequential,
    verbose=True  # ✅ boolean, no entero
)
```

### Variables de Entorno Requeridas

```bash
export OPENAI_BASE_URL="https://models.inference.ai.azure.com"
export GITHUB_TOKEN="tu_token_de_github_aquí"
```

## 🚀 Conclusiones

### Lo que hemos aprendido:

1. **CrewAI vs LangChain**: CrewAI orquesta equipos de agentes, LangChain proporciona los componentes individuales.

2. **Configuración crítica**: Cuando usas CrewAI con GitHub Models API, necesitas mapear las variables de entorno correctamente.

3. **Herramientas compatibles**: CrewAI requiere `BaseTool`, no el decorador `@tool` de LangChain.

4. **Parámetros correctos**: `verbose` debe ser boolean, no entero.

5. **Colaboración de agentes**: Los agentes pueden trabajar en secuencia, pasándose información entre ellos.

### Próximos pasos:

En los siguientes módulos exploraremos:
- Sistemas de memoria para agentes
- Integración con herramientas externas
- Estrategias de planificación más avanzadas
- Observabilidad y monitoreo de agentes

### 💡 Tip para estudiantes:

**Siempre revisa las configuraciones de entorno cuando cambies entre frameworks.** Cada framework tiene sus propias convenciones y expectativas sobre cómo acceder a los servicios externos. La clave está en entender estas diferencias y configurar correctamente las interfaces entre ellos.